## Hello there this is my gfdx notebook!

---


In [3]:
import os
from redcap import Project
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # progress bar
import dotdict
from dotenv import load_dotenv

load_dotenv()

# creates project using recap (PyCap) package initialization
project = Project(os.getenv("URL"), os.getenv("API_KEY"))


In [4]:
# gets ALL records from redcap project formatted as a dataframe
df = project.export_records(format="df")


In [5]:
# views first 10 rows of the dataframe
df.head(10)

redcap_repeat_instrument  \
country_code redcap_event_name                            
1.0          all_foods_arm_1                        NaN   
             maize_flour_arm_1                      NaN   
             oil_arm_1                              NaN   
             rice_arm_1                             NaN   
             salt_arm_1                             NaN   
             wheat_flour_arm_1                      NaN   
             all_foods_arm_2                        NaN   
             maize_flour_arm_2                      NaN   
             oil_arm_2                              NaN   
             rice_arm_2                             NaN   

                                redcap_repeat_instance country_name  \
country_code redcap_event_name                                        
1.0          all_foods_arm_1                       NaN      Armenia   
             maize_flour_arm_1                     NaN          NaN   
             oil_arm_1                             NaN          NaN   
             rice_arm_1                            NaN          NaN   
             salt_arm_1                            NaN          NaN   
             wheat_flour_arm_1                     NaN          NaN   
             all_foods_arm_2                       NaN      Armenia   
             maize_flour_arm_2                     NaN          NaN   
             oil_arm_2                             NaN          NaN   
             rice_arm_2                            NaN          NaN   

                               country_name_short  country_territory  \
country_code redcap_event_name                                         
1.0          all_foods_arm_1              Armenia                1.0   
             maize_flour_arm_1                NaN                NaN   
             oil_arm_1                        NaN                NaN   
             rice_arm_1                       NaN                NaN   
             salt_arm_1                       NaN                NaN   
             wheat_flour_arm_1                NaN                NaN   
             all_foods_arm_2              Armenia                1.0   
             maize_flour_arm_2                NaN                NaN   
             oil_arm_2                        NaN                NaN   
             rice_arm_2                       NaN                NaN   

                                un_region  ffi_region  unicef_region  \
country_code redcap_event_name                                         
1.0          all_foods_arm_1          3.0         4.0            NaN   
             maize_flour_arm_1        NaN         NaN            NaN   
             oil_arm_1                NaN         NaN            NaN   
             rice_arm_1               NaN         NaN            NaN   
             salt_arm_1               NaN         NaN            NaN   
             wheat_flour_arm_1        NaN         NaN            NaN   
             all_foods_arm_2          3.0         4.0            NaN   
             maize_flour_arm_2        NaN         NaN            NaN   
             oil_arm_2                NaN         NaN            NaN   
             rice_arm_2               NaN         NaN            NaN   

                                wb_income_status  wb_income_status_year  ...  \
country_code redcap_event_name                                           ...   
1.0          all_foods_arm_1                 NaN                    NaN  ...   
             maize_flour_arm_1               NaN                    NaN  ...   
             oil_arm_1                       NaN                    NaN  ...   
             rice_arm_1                      NaN                    NaN  ...   
             salt_arm_1                      NaN                    NaN  ...   
             wheat_flour_arm_1               NaN                    NaN  ...   
             all_foods_arm_2                 NaN                    NaN  ...   
             maize_

In [7]:
# shape of dataframe
rows, cols = df.shape
print(f"Dataframe has {rows} rows, and {cols} columns.")

(107253, 375)

In [ ]:
# target food list
food_list = [
    "maize_flour_arm_1",
    "wheat_flour_arm_1",
    "rice_arm_1",
    "salt_arm_1",
    "oil_arm_1",
]


In [ ]:
# filters dataframe for only target foods
filt_df = df[df.redcap_event_name.isin(food_list)]
countries = filt_df.country_code.unique()  # unique country codes
foods = filt_df.redcap_event_name.unique()  # unique food vehicles

In [ ]:
# declares targets during the loop
# targets are instruments and the corresponding column name
# formatted in a dict as instrument: instrument_value, col: column_name
targets = [
    {"instrument": "intake", "col": "food_intake"},
    {"instrument": "industrially_processed", "col": "industrially_processed_pc"},
    {"instrument": "coverage_fv", "col": "coverage_fv"},
]

In [ ]:
# initializes new, empty dataframe which will be appended to and re-written
new_df = pd.DataFrame()

# loops over each country, food, and target
for country in tqdm(countries):  # uses tqdm for nice progress bar
    for food in foods:
        for target in targets:
            # slices dataframe for only search rows 
            sliced = filt_df[
                (filt_df.country_code == country)
                & (filt_df.redcap_event_name == food)
                & (filt_df.redcap_repeat_instrument == target["instrument"])
            ]
            # gets most recent repeat_instance of the sliced dataframe
            # selects only desired columns
            single = sliced[
                sliced.redcap_repeat_instance == sliced.redcap_repeat_instance.max()
            ][
                [
                    "country_code",
                    "redcap_event_name",
                    "redcap_repeat_instrument",
                    target["col"],
                ]
            ]
            # if there were no matching rows we make a row using the target values
            # and manually input null into the target column
            if len(single) == 0:
                single_revised = pd.DataFrame(
                    [[country, food, target["instrument"], np.NaN]],
                    columns=[
                        "country_code",
                        "redcap_event_name",
                        "redcap_repeat_instrument",
                        target["col"],
                    ],
                )
                # append the new dataframe and reassign it to the original variable
                new_df = new_df.append(single_revised, ignore_index=True)
            else:
                # append the new dataframe and reassign it to the original variable
                new_df = new_df.append(single, ignore_index=True)


In [ ]:
# visualize new df
new_df.head(10)

In [ ]:
# initialize food groups in global space
grains = ["maize_flour_arm_1", "wheat_flour_arm_1", "rice_arm_1"]
oil = "oil_arm_1"
salt = "salt_arm_1"


In [ ]:

# TODO: document 
def identify_availability(row):
    return_val = ""
    if row.redcap_event_name in grains:
        # grains section
        if row.food_intake < 75:
            return_val = "Low"
        elif row.food_intake < 150:
            return_val = "Moderate"
        elif row.food_intake < 300:
            return_val = "Moderately High"
        elif row.food_intake >= 300:
            return_val = "High"
        else:
            return_val = "Unknown"
    elif row.redcap_event_name == oil:
        # oil section
        if row.food_intake < 20:
            return_val = "Low"
        elif row.food_intake < 40:
            return_val = "Moderate"
        elif row.food_intake < 60:
            return_val = "Moderately High"
        elif row.food_intake >= 60:
            return_val = "High"
        else:
            return_val = "Unknown"
    elif row.redcap_event_name == salt:
        # salt
        return_val = "High"
    else:
        # raises error message for invalid food vehicle
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val


# TODO: document 
def identify_coverage(row):
    return_val = ""
    if row.redcap_event_name in grains:
        # grains section
        if row.coverage_fv < 35:
            return_val = "Low"
        elif row.coverage_fv < 75:
            return_val = "Moderate"
        elif row.coverage_fv >= 75:
            return_val = "High"
        else:
            return_val = "Unknown"
    elif row.redcap_event_name in set([oil, salt]):
        # not grains section
        if row.coverage_fv < 35:
            return_val = "Low"
        elif row.coverage_fv < 75:
            return_val = "Moderate"
        else:
            return_val = "High"
    else:
        # raises error message for invalid food vehicle
        raise ValueError(f"Unexpected event_name {row.redcap_event_name!r}")
    return return_val

In [ ]:
# new columns apply functions to each row and return value for new column
new_df["food_avail"] = new_df.apply(lambda row: identify_availability(row), axis=1)
new_df["pop_coverage"] = new_df.apply(lambda row: identify_coverage(row), axis=1)
new_df.head()

,country_code,redcap_event_name,redcap_repeat_instrument,food_intake,industrially_processed_pc,coverage_fv,food_avail,pop_coverage
0,1.0,maize_flour_arm_1,intake,20.49,NaN,NaN,Low,Unknown
1,1.0,maize_flour_arm_1,industrially_processed,NaN,100.0,NaN,Unknown,Unknown
2,1.0,maize_flour_arm_1,coverage_fv,NaN,NaN,NaN,Unknown,Unknown
3,1.0,oil_arm_1,intake,25.32,NaN,NaN,Moderate,High
4,1.0,oil_arm_1,industrially_processed,NaN,NaN,NaN,Unknown,High


In [ ]:

# TODO: document and fix using recursion
# ? should this take a row?
def assess_reach(x, y):
    return_val = ""
    if x == "Low" and y == "Low":
        return_val = "Low"
    elif x == "Low" and y == "Moderate":
        return_val = "Moderate"
    elif x == "Low" and y == "High":
        return_val = "Moderate"
    elif x == "Moderate" and y == "Low":
        return_val = "Moderate"
    elif x == "Moderate" and y == "Moderate":
        return_val = "Moderate"
    elif x == "Moderate" and y == "High":
        return_val = "High"
    elif x == "Moderately High" and y == "Low":
        return_val = "Moderate"
    elif x == "Moderately High" and y == "Moderate":
        return_val = "Moderate"
    elif x == "Moderately High" and y == "High":
        return_val = "High"
    elif x == "High" and y == "Low":
        return_val = "Moderate"
    elif x == "High" and y == "Moderate":
        return_val = "High"
    elif x == "High" and y == "High":
        return_val = "High"
    elif x == "Unknown" and y == "Unknown":
        return_val = "Unknown"
    # recurse
    # elif x == 'Unknown' and y != 'Unknown':
    #     assess_reach(y, y)
    # # recurse
    # elif x != 'Unknown' and y == 'Unknown':
    #     assess_reach(x, x)
    # michi
    elif x == "Unknown" and y == "Low":
        return_val = "Low"
    elif x == "Unknown" and y == "Moderate":
        return_val = "Moderate"
    elif x == "Unknown" and y == "High":
        return_val = "High"
    elif x == "Low" and y == "Unknown":
        return_val = "Low"
    elif x == "Moderate" and y == "Unknown":
        return_val = "Moderate"
    elif x == "Moderately High" and y == "Unknown":
        return_val = "Moderate"
    elif x == "High" and y == "Unknown":
        return_val = "High"
    return return_val


# TODO: document and make take a row
def assess_industry(x):
    if x < 35:
        return "Significant"
    elif x < 75:
        return "Some"
    elif x >= 75:
        return "Minimal"
    else:
        return "Unknown"


In [ ]:
new_countries = []
new_foods = []
food_availabilities = []
industry_processes = []
pop_coverages = []
reaches = []

# loops over countries and foods and extracts values
# appends those values to above lists
for country in tqdm(new_df.country_code.unique()): # unique countries
    for food in new_df.redcap_event_name.unique(): # unique foods
        # temp df sliced for country, food
        temp = new_df[
            (new_df.country_code == country) & (new_df.redcap_event_name == food)
        ]
        # this section just gets desired values from specific rows
        food_avail = temp[temp.redcap_repeat_instrument == "intake"].food_avail.iloc[0]
        indust_proc = temp[
            temp.redcap_repeat_instrument == "industrially_processed"
        ].industrially_processed_pc.iloc[0]
        pop_cover = temp[
            temp.redcap_repeat_instrument == "coverage_fv"
        ].pop_coverage.iloc[0]

        # here we append those values to the corresponding lists
        new_countries.append(country)
        new_foods.append(food)
        food_availabilities.append(food_avail)
        industry_processes.append(assess_industry(indust_proc))
        pop_coverages.append(pop_cover)
        reaches.append(assess_reach(food_avail, pop_cover))


In [ ]:
# determines overall rating based on reach and industry processes
# TODO: document
def assess_overall(row):
    if row.reaches == "High" and row.industry_processes == "Minimal":
        return "Good"
    elif row.reaches == "High" and row.industry_processes == "Some":
        return "Good WC"
    elif row.reaches == "High" and row.industry_processes == "Significant":
        return "Moderate WC"
    elif row.reaches == "High" and row.industry_processes == "Unknown":
        return "Moderate WC"
    elif row.reaches == "Moderate" and row.industry_processes == "Minimal":
        return "Good"
    elif row.reaches == "Moderate" and row.industry_processes == "Some":
        return "Good WC"
    elif row.reaches == "Moderate" and row.industry_processes == "Significant":
        return "Moderate WC"
    elif row.reaches == "Moderate" and row.industry_processes == "Unknown":
        return "Moderate WC"
    elif row.reaches == "Low" and row.industry_processes == "Minimal":
        return "Moderate WC"
    elif row.reaches == "Low" and row.industry_processes == "Some":
        return "Moderate WC"
    elif row.reaches == "Low" and row.industry_processes == "Significant":
        return "Poor"
    elif row.reaches == "Low" and row.industry_processes == "Unknown":
        return "Poor"
    elif row.reaches == "Unknown" and row.industry_processes == "Minimal":
        return "Moderate"
    elif row.reaches == "Unknown" and row.industry_processes == "Some":
        return "Moderate"
    elif row.reaches == "Unknown" and row.industry_processes == "Significant":
        return "Poor"
    elif row.reaches == "Unknown" and row.industry_processes == "Unknown":
        return "Unknown"


In [ ]:
# make a new dataframe and columns using lists from above calculations
final = pd.DataFrame()
final["country_code"] = new_countries
final["foods"] = new_foods
final["food_avails"] = food_availabilities
final["industry_processes"] = industry_processes
final["pop_covers"] = pop_coverages
final["reaches"] = reaches
# apply overall algorithm
final["overall"] = final.apply(lambda row: assess_overall(row), axis=1)

final.head()

,country_code,foods,food_avails,industry_processes,pop_covers,reaches,overall
0,1.0,maize_flour_arm_1,Low,Minimal,Unknown,Low,Moderate WC
1,1.0,oil_arm_1,Moderate,Unknown,High,High,Moderate WC
2,1.0,rice_arm_1,Low,Minimal,Unknown,Low,Moderate WC
3,1.0,salt_arm_1,High,Unknown,High,High,Moderate WC
4,1.0,wheat_flour_arm_1,High,Minimal,Unknown,High,Good


In [ ]:
final

,country_code,foods,food_avails,industry_processes,pop_covers,reaches,overall
0,1.0,maize_flour_arm_1,Low,Minimal,Unknown,Low,Moderate WC
1,1.0,oil_arm_1,Moderate,Unknown,High,High,Moderate WC
2,1.0,rice_arm_1,Low,Minimal,Unknown,Low,Moderate WC
3,1.0,salt_arm_1,High,Unknown,High,High,Moderate WC
4,1.0,wheat_flour_arm_1,High,Minimal,Unknown,High,Good
...,...,...,...,...,...,...,...
1190,400.0,maize_flour_arm_1,Unknown,Some,Unknown,Unknown,Moderate
1191,400.0,oil_arm_1,Unknown,Unknown,High,High,Moderate WC
1192,400.0,rice_arm_1,Unknown,Minimal,Unknown,Unknown,Moderate
1193,400.0,salt_arm_1,High,Minimal,High,High,Good


In [ ]:
final.to_csv("i did this.csv")
